In [1]:
#generation debugging 
import sys
sys.path.insert(0, '/home/marcase/hgraph2graph/')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
import networkx as nx
import rdkit
import math, random
import numpy as np
import argparse
import os
from tqdm.auto import tqdm
import pickle
from hgraph import *
from hgraph.inc_graph import *
from hgraph.encoder import *
from hgraph.decoder import *
import matplotlib.pyplot as plt
from rdkit import DataStructs

/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# vocab = '/home/marcase/hgraph2graph/vocab23oct22.txt'
vocab = '/scratch/gthurber_root/gthurber0/marcase/preprocess_mono/preprocess_generated_real/vocab26oct22.txt'
vocab = [x.strip("\r\n ").split() for x in open(vocab)]
vocab = PairVocab(vocab)

In [3]:
class Args:
    train = '/scratch/gthurber_root/gthurber0/marcase/preprocess_mono/preprocess_generated_real/train/'
    vocab = vocab
    save_dir = 'test/'
    atom_vocab = common_atom_vocab
    load_model = None
    seed = 7
    rnn_type = 'LSTM'
    hidden_size=250
    embed_size=250
    batch_size=10
    latent_size=32
    depthT=15
    depthG=15
    diterT=1
    diterG=3
    dropout=0.0
    lr = 1e-3
    clip_norm=5.0
    step_beta=0.001
    max_beta=1.0
    warmup=10000
    kl_anneal_iter=2000
    epoch=20
    anneal_rate=0.9
    anneal_iter=25000
    print_iter=50
    save_iter=5000
    model = '/scratch/gthurber_root/gthurber0/marcase/preprocess_mono/preprocess_generated_real/ckpt/pretrained/model.ckpt.340000'
    load_model = True
    nsample = 20
    max_nodes=250
    max_edges=500
    max_AA = 15
    max_sub_nodes = 100
    label_size=2
    
args=Args()

model = HierVAE(args).cuda()

/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [4]:
print("Model #Params: %dK" % (sum([x.nelement() for x in model.parameters()]) / 1000,))

Model #Params: 17779K


In [5]:
#from generate
model.load_state_dict(torch.load(args.model)[0])
model.eval()
# torch.manual_seed(args.seed)
torch.manual_seed(np.random.randint(1,1000))
# random.seed(args.seed)
random.seed()

In [7]:
with torch.no_grad():
    for _ in tqdm(range(args.nsample // args.batch_size)):
        smiles_list = model.sample(args.batch_size, greedy=True,max_AA=args.max_AA)
        print(smiles_list)

  0%|          | 0/2 [00:00<?, ?it/s]

(2500, 2)
(2500, 38)


 50%|█████     | 1/2 [00:00<00:00,  1.04it/s]

['CSCCC1SC(Cc2ccc(O)cc2)C2NC(=O)C(Cc3ccc(O)cc3)NC(=O)C(CC(N)=O)NC(=O)C(CO)(NC(=O)CNC(=O)CNC(=O)C(CO)(NC(=O)C(N)CCC(N)=O)C(Cc3c[nH]c4ccccc34)SCC1=O)C(C)SC(CC(C)C)C(=O)C(CCCNC(=N)N)SCC(C(=O)N1CCCC1C(=O)O)NC(=O)CNC(=O)CNC2=O', 'CC1NC(=O)CNC(=O)C2CCCN2C(=O)C(NC(=O)C(N)CCC(N)=O)CSCC(=O)CSCC2NC(=O)C(Cc3c[nH]c4ccccc34)NC(=O)C(CSCC(=O)CSCC(C(=O)NC(CC(N)=O)C(=O)O)NC2=O)NC1=O', 'CC(O)C(N)C(=O)NC1CSCC(=O)CSCC(C(=O)NCC(=O)O)NC(=O)C(Cc2ccccc2)NC(=O)CNC(=O)C2CSCC(=O)CSCC(NC(=O)C(CCCNC(=N)N)NC(=O)CNC1=O)C(=O)NCC(=O)N2', 'NCCCCC1NC(=O)CNC(=O)C2CSCC(=O)CSCC(NC(=O)C(N)CCC(=O)O)C(=O)NCC(=O)NCC(=O)NC(CSCC(=O)CSCC(C(=O)NCC(=O)O)NC1=O)C(=O)N1CCCC1C(=O)N2', 'CC(C)CNCC(=O)O', 'NCCCCC(NC(=O)C(CO)NC(=O)C(NCCC(=O)O)C(CCC(N)=O)C(=O)O)C(=O)NC(NC1CSCC(=O)CSC(NCC(=O)O)C(C=O)(Cc2c[nH]c3ccccc23)NC(=O)C(Cc2ccccc2)NC(=O)CNC1=O)C(=O)CCC(N)=O', 'CSCCC1NC(=O)CNC(=O)C2CSCC(=O)CSCC(C(=O)N3CCCC3C(=O)O)NC(=O)C(Cc3ccc(O)cc3)NC(=O)CNC(=O)C(CSCC(=O)CSCC(NC(=O)C(N)CO)C(=O)NCC(=O)N2)NC(=O)C(Cc2cnc[nH]2)NC(=O)CNC1=O', 'CC(O)C(N)C(=O

100%|██████████| 2/2 [00:01<00:00,  1.18it/s]

['CC(C)CC(NC(=O)C1CSCC(=O)CSCC2NC(=O)CNC(=O)CNC(=O)C(NC(=O)C(N)CO)CSCC(=O)CSCC(NC(=O)C(Cc3ccc(O)cc3)NC2=O)C(=O)NCC(=O)NC(CCCNC(=N)N)C(=O)N1)C(=O)O', 'CSCCC(N)C(=O)NC1CSCC(=O)CSCC(C(=O)NC(Cc2ccccc2)C(=O)O)NC(=O)C2CSCC(=O)CSCC(NC(=O)C(CO)NC(=O)CNC(=O)C3CCCN3C1=O)C(=O)NC(C(C)C)C(=O)NC(Cc1ccc(O)cc1)C(=O)NC(C)C(=O)N2', 'NC(CO)C(=O)NC1CSCC(=O)CSCC(C(=O)NC(CO)C(=O)O)NC(=O)C(CCC(=O)O)NC(=O)CNC(=O)C2CSCC(=O)CSCC(NC(=O)C(Cc3c[nH]c4ccccc34)NC1=O)C(=O)NCC(=O)NCC(=O)NCC(=O)N2', 'N=C(N)NCCCC1NC(=O)C(NC(=O)CN)CSCC(=O)CSCC(C(=O)N2CCCC2C(=O)O)NC(=O)C(CCC(=O)O)NC(=O)CNC(=O)C2CSCC(=O)CSCC(NC1=O)C(=O)NC(Cc1ccc(O)cc1)C(=O)NC(CO)C(=O)NCC(=O)NCC(=O)N2', 'CCC(C)CC(CNC(C)C(=O)O)(Cc1cnc[nH]1)C(=O)C(N)C(C)O', 'CCC(C)C1NC(=O)C(Cc2ccc(O)cc2)NC(=O)C(NC(=O)C(Cc2c[nH]c3ccccc23)NC(=O)C2CSCC(=O)CSCC(NC(=O)C(N)CO)C(=O)NCC(=O)NC(CCC(=O)O)C(=O)N2)CSCC(=O)CSCC(C(=O)NC(CC(N)=O)C(=O)O)NC(=O)C(CCCNC(=N)N)NC1=O', 'CCC(C)C(NC(=O)C1CSCC(=O)CSCC(NC(=O)C2CSCC(=O)CSCC(NC(=O)C(N)CC(C)C)C(=O)NCC(=O)NCC(=O)NC(Cc3c[nH]c4ccccc34)C(=O)N2

In [29]:
#similarity
ms = [Chem.MolFromSmiles('CCOC'), Chem.MolFromSmiles('CCO'),Chem.MolFromSmiles('COC')]
fps = [Chem.RDKFingerprint(x) for x in ms]
DataStructs.FingerprintSimilarity(fps[0],fps[2],metric=DataStructs.TanimotoSimilarity)

0.4

In [35]:
#similarity - generated_molecules
ms = [Chem.MolFromSmiles(i) for i in smiles_list]
fps = [Chem.RDKFingerprint(x) for x in ms]
DataStructs.FingerprintSimilarity(fps[0],fps[4],metric=DataStructs.TanimotoSimilarity)

0.48890714872637636

In [47]:
Chem.MolToSmiles(Chem.ReplaceSubstructs(Chem.MolFromSmiles(smiles_list[1]), 
                                 Chem.MolFromSmiles('CSCC(CSC)=O'), 
                                 Chem.MolFromSmiles('F'),
                                 replaceAll=True)[0])

'CCC(C)C(NC(=O)CNC(=O)C(F)NC(=O)C(N)C(C)O)C(=O)NCC(=O)NCC(=O)NC(F)C(=O)NCC(=O)NC(Cc1ccccc1)C(=O)NCC(=O)NC(CO)C(=O)O'

In [48]:
def mol2seq(m):
    aa_smiles = {'ALA': 'C[C@H](N)C=O', 'CYS': 'N[C@H](C=O)CS', 'ASP': 'N[C@H](C=O)CC(=O)O', 'GLU': 'N[C@H](C=O)CCC(=O)O', 'PHE': 'N[C@H](C=O)Cc1ccccc1', 'GLY': 'NCC=O', 'HIS': 'N[C@H](C=O)Cc1c[nH]cn1', 'ILE': 'CC[C@H](C)[C@H](N)C=O', 'LYS': 'NCCCC[C@H](N)C=O', 'LEU': 'CC(C)C[C@H](N)C=O', 'MET': 'CSCC[C@H](N)C=O', 'ASN': 'NC(=O)C[C@H](N)C=O', 'PRO': 'O=C[C@@H]1CCCN1', 'GLN': 'NC(=O)CC[C@H](N)C=O', 'ARG': 'N=C(N)NCCC[C@H](N)C=O', 'SER': 'N[C@H](C=O)CO', 'THR': 'C[C@@H](O)[C@H](N)C=O', 'VAL': 'CC(C)[C@H](N)C=O', 'TRP': 'N[C@H](C=O)Cc1c[nH]c2ccccc12','TYR': 'N[C@H](C=O)Cc1ccc(O)cc1'}
    aas = ['GLY','ALA', 'VAL', 'CYS', 'ASP', 'GLU', 'PHE', 'HIS', 'ILE', 'LYS', 'LEU', 'MET', 'ASN', 'PRO', 'GLN', 'ARG', 'SER', 'THR', 'TRP','TYR'] #order important because gly is substructure of other aas
    # detect the atoms of the backbone and assign them with info
    CAatoms = m.GetSubstructMatches(Chem.MolFromSmarts("[C:0](=[O:1])[C:2][N:3]"))
    for atoms in CAatoms:
        a = m.GetAtomWithIdx(atoms[2])
        info = Chem.AtomPDBResidueInfo()
        info.SetName(" CA ") #spaces are important
        a.SetMonomerInfo(info)
    # detect the presence of residues and set residue name for CA atoms only
    for curr_aa in aas:
        matches = m.GetSubstructMatches(Chem.MolFromSmiles(aa_smiles[curr_aa]))
        for atoms in matches:
            for atom in atoms:
                a = m.GetAtomWithIdx(atom)
                info = Chem.AtomPDBResidueInfo()
                if a.GetMonomerInfo() != None:
                    if a.GetMonomerInfo().GetName() == " CA ":
                        info.SetName(" CA ")
                        info.SetResidueName(curr_aa)
                        a.SetMonomerInfo(info)
    # renumber the backbone atoms so the sequence order is correct:
    bbsmiles = "O"+"C(=O)CN"*len(m.GetSubstructMatches(Chem.MolFromSmiles(aa_smiles["GLY"]))) # generate backbone SMILES
    backbone = m.GetSubstructMatches(Chem.MolFromSmiles(bbsmiles))[0]
    id_list = list(backbone)
    id_list.reverse()
    for idx in [a.GetIdx() for a in m.GetAtoms()]:
        if idx not in id_list:
            id_list.append(idx)
    m_renum = Chem.RenumberAtoms(m,newOrder=id_list)
    return Chem.MolToSequence(m_renum)

In [51]:
test_mol = Chem.MolFromSmiles('CCC(C(C(NCC(NCC(NC(C(NCC(NC(C(NCC(NC(C(O)=O)CO)=O)=O)Cc1ccccc1)=O)=O)C)=O)=O)=O)NC(CNC(CNC(C(C(O)C)N)=O)=O)=O)C')
mol2seq(test_mol)

'TGGIGGAGFGS'

In [7]:
# #begin training the model
# for batch in tqdm(dataset):
#     total_step += 1
#     model.zero_grad()
#     loss, kl_div, wacc, iacc, tacc, sacc = model(*batch, beta=beta)

In [8]:
data_file0 = os.path.join(dataset.data_folder,'tensors-0.pkl')
with open(data_file0,'rb') as f:
    batches = pickle.load(f)


In [9]:
model.eval()
model(*batches[0],beta=beta)

(tensor(1.9945, device='cuda:0', grad_fn=<AddBackward0>),
 42.87664031982422,
 tensor(0.9921, device='cuda:0'),
 tensor(1., device='cuda:0'),
 tensor(1., device='cuda:0'),
 tensor(1., device='cuda:0'))

In [10]:
# nx.draw(batches[0][0][1])
# plt.show()

#just realized I have no idea what format the tensorized molecules are in. let's go back and check the tensorize function

In [11]:
#from model.sample()
batch_size = args.batch_size
latent_size = args.latent_size
root_vecs = torch.randn(batch_size, latent_size).cuda()

In [12]:
#from HierMPNDecoder.decode()
src_root_vecs, src_tree_vecs, src_graph_vecs = root_vecs,root_vecs,root_vecs
batch_size = len(src_root_vecs)
batch_size

1

In [13]:
tree_batch = IncTree(batch_size, node_fdim=2, edge_fdim=3)
# nx.draw(tree_batch.graph)

(100, 2)


In [14]:
hmpn = IncHierMPNEncoder(args.vocab, args.atom_vocab, args.rnn_type, args.embed_size, args.hidden_size, args.depthT, args.depthG, args.dropout)
test = hmpn.tree_encoder
test.depth

15

In [15]:
graph_batch = IncGraph(args.atom_vocab, batch_size, node_fdim=hmpn.atom_size, edge_fdim=hmpn.atom_size + hmpn.bond_size)
# nx.draw(graph_batch.graph)

(100, 38)


In [16]:
def get_cls_score(src_tree_vecs, batch_idx, cls_vecs, cls_labs):
    use_attention = False
    if use_attention:
        cls_cxt = self.attention(src_tree_vecs, batch_idx, cls_vecs, self.A_cls)
    else:
        cls_cxt = src_tree_vecs.index_select(index=batch_idx, dim=0).cuda()

    cls_vecs = torch.cat([cls_vecs, cls_cxt], dim=-1).cuda()
    clsNN = nn.Sequential(
                nn.Linear(args.hidden_size + args.latent_size, args.hidden_size),
                nn.ReLU(),
                nn.Dropout(args.dropout),
                nn.Linear(args.hidden_size, args.vocab.size()[0])
        ).cuda()
    iclsNN = nn.Sequential(
                nn.Linear(args.hidden_size + args.latent_size, args.hidden_size),
                nn.ReLU(),
                nn.Dropout(args.dropout),
                nn.Linear(args.hidden_size, args.vocab.size()[1])
        ).cuda()
    cls_scores = clsNN(cls_vecs)

    if cls_labs is None: #inference mode
        icls_scores = iclsNN(cls_vecs) #no masking
    else:
        vocab_masks = args.vocab.get_mask(cls_labs)
        icls_scores = iclsNN(cls_vecs) + vocab_masks #apply mask by log(x + mask): mask=0 or -INF
    return cls_scores, icls_scores

In [17]:
stack = [[] for i in range(batch_size)]
W_root = nn.Linear(args.latent_size, args.hidden_size).cuda()
init_vecs = src_root_vecs if args.latent_size == args.hidden_size else W_root(src_root_vecs)
itensor = torch.LongTensor([]).cuda()
batch_idx = itensor.new_tensor(range(batch_size))
cls_scores, icls_scores = get_cls_score(src_tree_vecs, batch_idx, init_vecs, None)
root_cls = cls_scores.max(dim=-1)[1]
icls_scores = icls_scores + args.vocab.get_mask(root_cls)
root_cls, root_icls = root_cls.tolist(), icls_scores.max(dim=-1)[1].tolist()

In [18]:
super_root = tree_batch.add_node() 
for bid in range(batch_size):
    clab, ilab = root_cls[bid], root_icls[bid]
    root_idx = tree_batch.add_node( batch_idx.new_tensor([clab, ilab]) )
    tree_batch.add_edge(super_root, root_idx) 
    stack[bid].append(root_idx)

    root_smiles = args.vocab.get_ismiles(ilab)
    new_atoms, new_bonds, attached = graph_batch.add_mol(bid, root_smiles, [], 0)
    tree_batch.register_cgraph(root_idx, new_atoms, new_bonds, attached)

In [19]:
# nx.draw(tree_batch.graph)

In [20]:
class HTuple():
    def __init__(self, node=None, mess=None, vmask=None, emask=None):
        self.node, self.mess = node, mess
        self.vmask, self.emask = vmask, emask
#invariance: tree_tensors is equal to inter_tensors (but inter_tensor's init_vec is 0)
tree_tensors = tree_batch.get_tensors()
graph_tensors = graph_batch.get_tensors()

htree = HTuple( mess = hmpn.tree_encoder.rnn.get_init_state(tree_tensors[1]) )
hinter = HTuple( mess = hmpn.tree_encoder.rnn.get_init_state(tree_tensors[1]) )
hgraph = HTuple( mess = hmpn.tree_encoder.rnn.get_init_state(graph_tensors[1]) )
h = hmpn.tree_encoder.rnn.get_hidden_state(htree.mess)
h[1 : batch_size + 1] = init_vecs #wiring root (only for tree, not inter)

In [21]:
#decode one step at a time
t=0 #up to max_decode_step, which is 100 by default
batch_list = [ bid for bid in range(batch_size) if len(stack[bid]) > 0 ]

In [22]:
batch_idx = batch_idx.new_tensor(batch_list)
cur_tree_nodes = [stack[bid][-1] for bid in batch_list]
subtree = batch_idx.new_tensor(cur_tree_nodes).cuda(), batch_idx.new_tensor([]).cuda()
subgraph = batch_idx.new_tensor( tree_batch.get_cluster_nodes(cur_tree_nodes) ).cuda(), batch_idx.new_tensor( tree_batch.get_cluster_edges(cur_tree_nodes) ).cuda()

In [24]:
htree, hinter, hgraph = hmpn(tree_tensors, tree_tensors, graph_tensors, htree, hinter, hgraph, subtree, subgraph)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_addmm)

In [ ]:
# sub_graph_tensors = hmpn.get_sub_tensor(graph_tensors,subgraph)[:-1]
# fnode, fmess, agraph, bgraph = sub_graph_tensors 
# subnode, submess = subgraph
# # if len(submess) > 0: 
# #     h = hmpn.graph_encoder.rnn.sparse_forward(hgraph.mess, fmess, submess, bgraph)
# atom_size = common_atom_vocab.size()
# bond_size = len(MolGraph.BOND_LIST) + MolGraph.MAX_POS
# rnn = LSTM(atom_size + bond_size, args.hidden_size, args.depthG)
# bgraph
# # h = rnn(fmess, bgraph)
# h = torch.zeros(fmess.size(0), args.hidden_size, device=fmess.device)
# c = torch.zeros(fmess.size(0), args.hidden_size, device=fmess.device)
# mask = torch.ones(h.size(0), 1, device=h.device)
# mask[0, 0] = 0 #first message is padding

# for i in range(args.depthG):
#     h_nei = index_select_ND(h, 0, bgraph)
#     c_nei = index_select_ND(c, 0, bgraph)
#     h,c = LSTM(fmess, h_nei, c_nei)
#     h = h * mask
#     c = c * mask
# # h = rnn.get_hidden_state(h)
# # h,c = h
# # mask = h.new_ones(h.size(0)).scatter_(0, submess, 0)
# # h = h * mask.unsqueeze(1)
# # c = c * mask.unsqueeze(1)

In [24]:
for i in range(args.depthG):
    h_nei = index_select_ND(h, 0, bgraph)
    c_nei = index_select_ND(c, 0, bgraph)

In [ ]:
def get_topo_score(self, src_tree_vecs, batch_idx, topo_vecs):
    use_attention = False
    if use_attention:
        topo_cxt = self.attention(src_tree_vecs, batch_idx, topo_vecs, self.A_topo)
    else:
        topo_cxt = src_tree_vecs.index_select(index=batch_idx, dim=0)
    return self.topoNN( torch.cat([topo_vecs, topo_cxt], dim=-1) ).squeeze(-1)
topo_scores = get_topo_score(src_tree_vecs, batch_idx, htree.node.index_select(0, subtree[0]))
topo_scores = torch.sigmoid(topo_scores)
if greedy:
    topo_preds = topo_scores.tolist()
else:
    topo_preds = torch.bernoulli(topo_scores).tolist()